# Les 8: Kansrekening en Kansdichtheid

**Mathematical Foundations - IT & Artificial Intelligence**

---

## 8.0 Recap en Motivatie

In Deel 1 en 2 hebben we geleerd hoe neurale netwerken data verwerken (lineaire algebra) en hoe ze leren (calculus). We kunnen nu een netwerk trainen dat met ~97% nauwkeurigheid cijfers herkent.

Maar wat betekent de output van een neuraal netwerk eigenlijk? Als we softmax gebruiken, krijgen we getallen tussen 0 en 1 die optellen tot 1. Dit zijn **kansen**! Het netwerk voorspelt de kans dat een input tot elke klasse behoort.

In Deel 3 duiken we in de statistiek en kansrekening die ten grondslag ligt aan machine learning:

- **Les 8**: Kansrekening - de taal van onzekerheid
- **Les 9**: Verwachtingswaarde en variantie - samenvattende statistieken
- **Les 10**: Maximum Likelihood - de theoretische basis van training

Dit geeft ons een dieper begrip van waarom we bepaalde loss functies gebruiken en hoe we de output van een model moeten interpreteren.

## 8.1 Leerdoelen

Na deze les begrijp je de basisprincipes van kansrekening. Je kunt werken met discrete en continue kansverdelingen. Je begrijpt conditionele kans en de regel van Bayes. Je kunt de normale verdeling toepassen. Je begrijpt hoe softmax output als kansverdeling werkt.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

np.set_printoptions(precision=4, suppress=True)
np.random.seed(42)

print("Libraries geladen!")

## 8.2 Basisprincipes van Kansrekening

### Wat is kans?

Kans is een maat voor onzekerheid. Het geeft aan hoe waarschijnlijk een bepaalde uitkomst is. Kansen liggen altijd tussen 0 (onmogelijk) en 1 (zeker).

### Kansruimte

Een kansruimte bestaat uit:
- **Uitkomstenruimte Ω**: alle mogelijke uitkomsten
- **Gebeurtenissen**: deelverzamelingen van Ω
- **Kansfunctie P**: wijst aan elke gebeurtenis een kans toe

### Axioma's van Kolmogorov

1. P(A) ≥ 0 voor elke gebeurtenis A
2. P(Ω) = 1 (iets moet gebeuren)
3. P(A ∪ B) = P(A) + P(B) als A en B disjunct zijn

In [ ]:
# Voorbeeld: eerlijke dobbelsteen
# Uitkomstenruimte Ω = {1, 2, 3, 4, 5, 6}

outcomes = [1, 2, 3, 4, 5, 6]
probabilities = [1/6] * 6  # Eerlijke dobbelsteen

print("Eerlijke dobbelsteen:")
for outcome, prob in zip(outcomes, probabilities):
    print(f"  P(X = {outcome}) = {prob:.4f}")

print(f"\nSom van kansen: {sum(probabilities)}")

# Visualisatie
plt.figure(figsize=(8, 5))
plt.bar(outcomes, probabilities, color='steelblue', edgecolor='black')
plt.xlabel('Uitkomst', fontsize=12)
plt.ylabel('Kans', fontsize=12)
plt.title('Kansverdeling van een eerlijke dobbelsteen', fontsize=14)
plt.ylim(0, 0.3)
plt.xticks(outcomes)
plt.grid(axis='y', alpha=0.3)
plt.show()

In [ ]:
# Simulatie: gooi de dobbelsteen 10000 keer
n_throws = 10000
throws = np.random.randint(1, 7, n_throws)

# Tel frequenties
counts = [np.sum(throws == i) for i in range(1, 7)]
frequencies = [c / n_throws for c in counts]

print(f"Simulatie van {n_throws} worpen:")
for outcome, freq in zip(outcomes, frequencies):
    print(f"  Frequentie van {outcome}: {freq:.4f} (theoretisch: {1/6:.4f})")

# Wet van grote aantallen: frequentie nadert kans
plt.figure(figsize=(10, 5))

# Cumulatieve frequentie van "6" na elke worp
is_six = (throws == 6).astype(float)
cumulative_freq = np.cumsum(is_six) / np.arange(1, n_throws + 1)

plt.plot(cumulative_freq, 'b-', alpha=0.7)
plt.axhline(y=1/6, color='r', linestyle='--', label='Theoretische kans = 1/6')
plt.xlabel('Aantal worpen', fontsize=12)
plt.ylabel('Frequentie van 6', fontsize=12)
plt.title('Wet van Grote Aantallen: frequentie convergeert naar kans', fontsize=14)
plt.legend(fontsize=11)
plt.xscale('log')
plt.grid(True, alpha=0.3)
plt.show()

## 8.3 Discrete Kansverdelingen

Een **discrete kansverdeling** beschrijft de kansen van een variabele die alleen bepaalde waarden kan aannemen (zoals gehele getallen).

### Kansfunctie (PMF)

De probability mass function P(X = x) geeft de kans dat X gelijk is aan x.

Eigenschappen:
- P(X = x) ≥ 0 voor alle x
- Σ P(X = x) = 1

### Belangrijke discrete verdelingen

**Bernoulli verdeling**: één experiment met twee uitkomsten (succes/faling)
- P(X = 1) = p, P(X = 0) = 1-p

**Binomiaal verdeling**: aantal successen in n onafhankelijke Bernoulli trials
- P(X = k) = C(n,k) · p^k · (1-p)^(n-k)

**Categorische verdeling**: één experiment met K mogelijke uitkomsten
- Dit is wat softmax output representeert!

In [ ]:
# Binomiaal verdeling: aantal keer kop bij 10 muntworpen
n = 10  # aantal worpen
p = 0.5  # kans op kop

k_values = np.arange(0, n + 1)
binomial_probs = stats.binom.pmf(k_values, n, p)

plt.figure(figsize=(10, 5))
plt.bar(k_values, binomial_probs, color='steelblue', edgecolor='black')
plt.xlabel('Aantal keer kop (k)', fontsize=12)
plt.ylabel('P(X = k)', fontsize=12)
plt.title(f'Binomiaal verdeling: n={n}, p={p}', fontsize=14)
plt.xticks(k_values)
plt.grid(axis='y', alpha=0.3)
plt.show()

print(f"P(X = 5) = {stats.binom.pmf(5, n, p):.4f}")
print(f"P(X ≥ 7) = {1 - stats.binom.cdf(6, n, p):.4f}")

In [ ]:
# Categorische verdeling: softmax output!
# Dit is precies wat een classifier voorspelt

# Stel: netwerk output voor een afbeelding van een "7"
logits = np.array([0.1, -0.5, 0.3, 0.2, -0.1, 0.5, -0.3, 2.5, 0.1, 0.0])

def softmax(x):
    exp_x = np.exp(x - np.max(x))
    return exp_x / np.sum(exp_x)

probs = softmax(logits)

print("Softmax output = categorische kansverdeling:")
for digit, prob in enumerate(probs):
    bar = '█' * int(prob * 50)
    print(f"  P(digit = {digit}) = {prob:.4f} {bar}")

print(f"\nSom van kansen: {np.sum(probs):.6f}")
print(f"Voorspelling: {np.argmax(probs)} (hoogste kans)")

In [ ]:
# Visualisatie
plt.figure(figsize=(10, 5))
colors = ['steelblue'] * 10
colors[7] = 'orangered'  # Highlight de voorspelling

plt.bar(range(10), probs, color=colors, edgecolor='black')
plt.xlabel('Digit', fontsize=12)
plt.ylabel('Kans', fontsize=12)
plt.title('Softmax output als categorische kansverdeling', fontsize=14)
plt.xticks(range(10))
plt.grid(axis='y', alpha=0.3)
plt.show()

print("De softmax output is een geldige kansverdeling:")
print("- Alle waarden ≥ 0 ✓")
print("- Som = 1 ✓")

## 8.4 Continue Kansverdelingen

Een **continue kansverdeling** beschrijft de kansen van een variabele die elke waarde in een interval kan aannemen.

### Kansdichtheidsfunctie (PDF)

Voor continue variabelen gebruiken we een probability density function f(x). De kans dat X in een interval [a, b] valt is:

P(a ≤ X ≤ b) = ∫[a,b] f(x) dx

Let op: f(x) zelf is geen kans! Het is een dichtheid. P(X = exact x) = 0 voor continue variabelen.

### Cumulatieve Distributiefunctie (CDF)

F(x) = P(X ≤ x) = ∫[-∞,x] f(t) dt

De CDF geeft de kans dat X kleiner of gelijk is aan x.

In [ ]:
# Uniforme verdeling op [0, 1]
x = np.linspace(-0.5, 1.5, 1000)

# PDF
pdf_uniform = np.where((x >= 0) & (x <= 1), 1, 0)

# CDF
cdf_uniform = np.clip(x, 0, 1)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(x, pdf_uniform, 'b-', linewidth=2)
axes[0].fill_between(x, pdf_uniform, alpha=0.3)
axes[0].set_xlabel('x', fontsize=12)
axes[0].set_ylabel('f(x)', fontsize=12)
axes[0].set_title('PDF: Uniforme verdeling U(0,1)', fontsize=12)
axes[0].set_ylim(-0.1, 1.5)
axes[0].grid(True, alpha=0.3)

axes[1].plot(x, cdf_uniform, 'r-', linewidth=2)
axes[1].set_xlabel('x', fontsize=12)
axes[1].set_ylabel('F(x) = P(X ≤ x)', fontsize=12)
axes[1].set_title('CDF: Uniforme verdeling U(0,1)', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("P(0.3 ≤ X ≤ 0.7) = 0.7 - 0.3 = 0.4")
print("Dit is de oppervlakte onder de PDF tussen 0.3 en 0.7")

## 8.5 De Normale Verdeling

De **normale verdeling** (Gaussische verdeling) is de belangrijkste verdeling in statistiek en machine learning.

### Formule

f(x) = (1 / √(2πσ²)) · exp(-(x-μ)² / (2σ²))

waarbij:
- μ (mu) = gemiddelde (centrum van de verdeling)
- σ (sigma) = standaarddeviatie (breedte van de verdeling)

### Waarom zo belangrijk?

1. **Centrale Limietstelling**: het gemiddelde van veel onafhankelijke variabelen is normaal verdeeld
2. **Maximum entropie**: gegeven gemiddelde en variantie, is de normale verdeling de "meest onzekere"
3. **Analytisch handig**: integralen en afgeleiden zijn bekend

### In Machine Learning

- Weight initialisatie (vaak normaal verdeeld)
- Gaussian noise voor regularisatie
- Variational autoencoders
- Gaussian processes

In [ ]:
# Normale verdeling met verschillende parameters
x = np.linspace(-6, 6, 1000)

plt.figure(figsize=(12, 5))

# Verschillende μ en σ
params = [(0, 1, 'Standaard: μ=0, σ=1'),
          (0, 0.5, 'Smaller: μ=0, σ=0.5'),
          (0, 2, 'Wider: μ=0, σ=2'),
          (2, 1, 'Shifted: μ=2, σ=1')]

for mu, sigma, label in params:
    pdf = stats.norm.pdf(x, mu, sigma)
    plt.plot(x, pdf, linewidth=2, label=label)

plt.xlabel('x', fontsize=12)
plt.ylabel('f(x)', fontsize=12)
plt.title('Normale verdelingen met verschillende parameters', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# De 68-95-99.7 regel
mu, sigma = 0, 1
x = np.linspace(-4, 4, 1000)
pdf = stats.norm.pdf(x, mu, sigma)

fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(x, pdf, 'b-', linewidth=2)

# Vul gebieden in
ax.fill_between(x, pdf, where=(x >= -1) & (x <= 1), alpha=0.3, color='green', label='68% (±1σ)')
ax.fill_between(x, pdf, where=((x >= -2) & (x < -1)) | ((x > 1) & (x <= 2)), alpha=0.3, color='yellow', label='95% (±2σ)')
ax.fill_between(x, pdf, where=((x >= -3) & (x < -2)) | ((x > 2) & (x <= 3)), alpha=0.3, color='red', label='99.7% (±3σ)')

ax.set_xlabel('x (in standaarddeviaties)', fontsize=12)
ax.set_ylabel('f(x)', fontsize=12)
ax.set_title('De 68-95-99.7 regel voor de normale verdeling', fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.show()

print("68-95-99.7 regel:")
print(f"  P(-1 ≤ X ≤ 1) = {stats.norm.cdf(1) - stats.norm.cdf(-1):.4f} ≈ 68%")
print(f"  P(-2 ≤ X ≤ 2) = {stats.norm.cdf(2) - stats.norm.cdf(-2):.4f} ≈ 95%")
print(f"  P(-3 ≤ X ≤ 3) = {stats.norm.cdf(3) - stats.norm.cdf(-3):.4f} ≈ 99.7%")

In [ ]:
# Centrale Limietstelling demonstratie
# Gemiddelde van n uniforme variabelen wordt normaal verdeeld

fig, axes = plt.subplots(2, 3, figsize=(15, 8))

n_samples = 10000

for ax, n in zip(axes.flatten(), [1, 2, 5, 10, 30, 100]):
    # Genereer n uniforme samples en neem het gemiddelde
    samples = np.random.uniform(0, 1, (n_samples, n))
    means = np.mean(samples, axis=1)
    
    ax.hist(means, bins=50, density=True, alpha=0.7, color='steelblue', edgecolor='black')
    
    # Theoretische normale verdeling
    mu = 0.5  # E[U(0,1)] = 0.5
    sigma = np.sqrt(1/12) / np.sqrt(n)  # Var[U(0,1)] = 1/12
    x = np.linspace(0, 1, 100)
    ax.plot(x, stats.norm.pdf(x, mu, sigma), 'r-', linewidth=2, label='Normale benadering')
    
    ax.set_title(f'n = {n}')
    ax.set_xlim(0, 1)

plt.suptitle('Centrale Limietstelling: gemiddelde van n uniforme variabelen', fontsize=14)
plt.tight_layout()
plt.show()

print("Hoe meer variabelen we middelen, hoe meer het gemiddelde")
print("lijkt op een normale verdeling!")

## 8.6 Conditionele Kans en Bayes

### Conditionele kans

De kans op A gegeven dat B is gebeurd:

P(A|B) = P(A ∩ B) / P(B)

### Regel van Bayes

De regel van Bayes laat ons toe om "om te keren":

P(A|B) = P(B|A) · P(A) / P(B)

In ML termen:
- P(A) = **prior**: wat we geloofden vóór we data zagen
- P(B|A) = **likelihood**: hoe waarschijnlijk is de data gegeven onze hypothese
- P(A|B) = **posterior**: wat we geloven ná het zien van de data

### Waarom belangrijk voor ML?

Classificatie kan gezien worden als: gegeven de input x, wat is P(klasse|x)?

In [ ]:
# Voorbeeld: medische test
# Een test voor een ziekte heeft:
# - 99% sensitiviteit: P(positief | ziek) = 0.99
# - 95% specificiteit: P(negatief | gezond) = 0.95
# De ziekte komt voor bij 1% van de bevolking: P(ziek) = 0.01

# Vraag: als je positief test, wat is de kans dat je echt ziek bent?
# Dit is P(ziek | positief)

P_ziek = 0.01
P_gezond = 1 - P_ziek
P_pos_gegeven_ziek = 0.99  # Sensitiviteit
P_neg_gegeven_gezond = 0.95  # Specificiteit
P_pos_gegeven_gezond = 1 - P_neg_gegeven_gezond  # False positive rate

# P(positief) = P(pos|ziek)·P(ziek) + P(pos|gezond)·P(gezond)
P_positief = P_pos_gegeven_ziek * P_ziek + P_pos_gegeven_gezond * P_gezond

# Bayes: P(ziek|positief) = P(pos|ziek)·P(ziek) / P(positief)
P_ziek_gegeven_pos = (P_pos_gegeven_ziek * P_ziek) / P_positief

print("Medische test voorbeeld:")
print(f"  P(ziek) = {P_ziek} (1% van bevolking)")
print(f"  P(positief | ziek) = {P_pos_gegeven_ziek} (sensitiviteit)")
print(f"  P(negatief | gezond) = {P_neg_gegeven_gezond} (specificiteit)")
print()
print(f"  P(positief) = {P_positief:.4f}")
print(f"  P(ziek | positief) = {P_ziek_gegeven_pos:.4f} = {P_ziek_gegeven_pos*100:.1f}%")
print()
print("Verrassend! Zelfs met een positieve test is de kans op ziekte slechts ~17%.")
print("Dit komt door de lage base rate (1% is ziek).")

In [ ]:
# Visualisatie met natuurlijke frequenties
# Stel 10000 mensen worden getest

n_people = 10000
n_sick = int(n_people * P_ziek)  # 100 ziek
n_healthy = n_people - n_sick    # 9900 gezond

n_sick_pos = int(n_sick * P_pos_gegeven_ziek)  # 99 ziek en positief
n_healthy_pos = int(n_healthy * P_pos_gegeven_gezond)  # 495 gezond maar positief

print(f"Van {n_people} mensen:")
print(f"  {n_sick} zijn ziek, {n_healthy} zijn gezond")
print(f"  {n_sick_pos} zieken testen positief")
print(f"  {n_healthy_pos} gezonden testen positief (vals positief!)")
print(f"  Totaal positief: {n_sick_pos + n_healthy_pos}")
print(f"  Kans ziek gegeven positief: {n_sick_pos}/{n_sick_pos + n_healthy_pos} = {n_sick_pos/(n_sick_pos + n_healthy_pos):.4f}")

# Visualisatie
fig, ax = plt.subplots(figsize=(10, 6))

categories = ['Ziek\nPositief', 'Ziek\nNegatief', 'Gezond\nPositief', 'Gezond\nNegatief']
counts = [n_sick_pos, n_sick - n_sick_pos, n_healthy_pos, n_healthy - n_healthy_pos]
colors = ['red', 'lightcoral', 'orange', 'lightgreen']

ax.bar(categories, counts, color=colors, edgecolor='black')
for i, (cat, count) in enumerate(zip(categories, counts)):
    ax.text(i, count + 100, str(count), ha='center', fontsize=11)

ax.set_ylabel('Aantal mensen', fontsize=12)
ax.set_title('Uitkomsten van 10,000 tests', fontsize=14)
ax.set_ylim(0, 10000)
plt.show()

## 8.7 Toepassing: Classifier Output als Kans

In neurale netwerken voor classificatie:
- De softmax output is een kansverdeling over klassen
- We trainen het netwerk om P(klasse | input) te modelleren
- Cross-entropy loss meet hoe goed deze kansverdeling past bij de werkelijke labels

In [ ]:
# Simuleer classifier output voor MNIST
# Goed gekalibreerde vs slecht gekalibreerde classifier

def softmax(x):
    exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

# Simuleer 1000 voorspellingen
np.random.seed(42)
n_samples = 1000

# True labels (uniform verdeeld over 10 klassen)
true_labels = np.random.randint(0, 10, n_samples)

# Goed model: hoge confidence voor correcte klasse
good_logits = np.random.randn(n_samples, 10) * 0.5
good_logits[np.arange(n_samples), true_labels] += 3  # Boost correcte klasse
good_probs = softmax(good_logits)

# Slecht model: lage confidence, meer uniform
bad_logits = np.random.randn(n_samples, 10) * 0.5
bad_logits[np.arange(n_samples), true_labels] += 0.5  # Kleine boost
bad_probs = softmax(bad_logits)

# Confidence van het model (max probability)
good_confidence = np.max(good_probs, axis=1)
bad_confidence = np.max(bad_probs, axis=1)

# Correctheid
good_correct = (np.argmax(good_probs, axis=1) == true_labels)
bad_correct = (np.argmax(bad_probs, axis=1) == true_labels)

print(f"Goed model: {np.mean(good_correct)*100:.1f}% accuracy, gem. confidence: {np.mean(good_confidence):.3f}")
print(f"Slecht model: {np.mean(bad_correct)*100:.1f}% accuracy, gem. confidence: {np.mean(bad_confidence):.3f}")

In [ ]:
# Calibration plot: is de confidence gerelateerd aan de accuracy?
def calibration_plot(probs, labels, model_name, ax):
    confidence = np.max(probs, axis=1)
    predictions = np.argmax(probs, axis=1)
    correct = (predictions == labels)
    
    # Bin confidence in 10 bins
    bins = np.linspace(0, 1, 11)
    bin_indices = np.digitize(confidence, bins) - 1
    bin_indices = np.clip(bin_indices, 0, 9)
    
    bin_accuracies = []
    bin_confidences = []
    
    for i in range(10):
        mask = (bin_indices == i)
        if np.sum(mask) > 0:
            bin_accuracies.append(np.mean(correct[mask]))
            bin_confidences.append(np.mean(confidence[mask]))
        else:
            bin_accuracies.append(0)
            bin_confidences.append((bins[i] + bins[i+1]) / 2)
    
    ax.bar(np.arange(10) * 0.1 + 0.05, bin_accuracies, width=0.08, alpha=0.7, label='Accuracy')
    ax.plot([0, 1], [0, 1], 'r--', label='Perfect calibration')
    ax.set_xlabel('Confidence')
    ax.set_ylabel('Accuracy')
    ax.set_title(f'{model_name}')
    ax.legend()
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.grid(True, alpha=0.3)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))
calibration_plot(good_probs, true_labels, 'Goed Model', axes[0])
calibration_plot(bad_probs, true_labels, 'Slecht Model', axes[1])
plt.suptitle('Calibratie: komt confidence overeen met accuracy?', fontsize=14)
plt.tight_layout()
plt.show()

print("Een goed gekalibreerd model: als het 80% confident is, is het ~80% van de tijd correct.")

## 8.8 Samenvatting

### Kernconcepten

**Kansrekening** is de wiskundige taal van onzekerheid. Kansen liggen tussen 0 en 1, en sommeren tot 1.

**Discrete verdelingen** beschrijven variabelen met aftelbare uitkomsten. De categorische verdeling is precies wat softmax output representeert.

**Continue verdelingen** gebruiken een dichtheidsfunctie. De normale verdeling is de belangrijkste, dankzij de centrale limietstelling.

**Bayes' regel** laat ons toe om kansen te updaten op basis van nieuwe informatie. Dit is de basis van probabilistisch redeneren.

### Link naar neurale netwerken

- Softmax output = categorische kansverdeling
- Het netwerk leert P(klasse | input) te modelleren
- Calibratie: komt de confidence overeen met de werkelijke accuracy?

### Volgende les

In les 9 leren we over verwachtingswaarde en variantie: hoe vatten we een verdeling samen in enkele getallen?

---

**Mathematical Foundations** | Les 8 van 12 | IT & Artificial Intelligence

---